In [1]:
!pip show google-generativeai
!pip install --upgrade google-generativeai
!pip install PyPDF2
!pip install flask

Name: google-generativeai
Version: 0.8.3
Summary: Google Generative AI High level API client library and tools.
Home-page: https://github.com/google/generative-ai-python
Author: Google LLC
Author-email: googleapis-packages@google.com
License: Apache 2.0
Location: /opt/app-root/lib/python3.9/site-packages
Requires: google-ai-generativelanguage, google-api-core, google-api-python-client, google-auth, protobuf, pydantic, tqdm, typing-extensions
Required-by: 

[notice] A new release of pip available: 22.2.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.4/224.4 kB 19.5 MB/s eta 0:00:00

[notice] A new release of pip available: 22.2.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
from flask import Flask, request, jsonify
import google.generativeai as genai
import PyPDF2
import re
import os

In [3]:

genai.configure(api_key="********")

model = genai.GenerativeModel('gemini-pro')

app = Flask(__name__)
app.config['UPLOAD_FOLDER'] = 'srs.pdf'

# Function to extract text from a PDF document
def extract_text_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as pdf_file:
        pdf_reader = PyPDF2.PdfReader(pdf_file)
        extracted_text = ""
        for page in pdf_reader.pages:
            text = page.extract_text()
            if text:
                extracted_text += text
        return extracted_text

# Function to parse user stories from extracted text
def parse_user_stories(text):
    user_stories = {}
    matches = re.findall(r'(User Story \d+:)(.*?)(?=User Story \d+:|$)', text, re.DOTALL)
    for match in matches:
        story_id, story_text = match[0].strip(), match[1].strip()
        user_stories[story_id] = story_text
    return user_stories

# Function to generate either an effort estimate or Node.js implementation guide
def generate_response(user_story, mode="estimate"):
    if mode == "estimate":
        prompt = f"Estimate the development effort in hours required for the following user story:\n\n{user_story}"
    elif mode == "guide":
        prompt = f"""
        Provide a detailed guide on how to implement the following user story as a backend service using Node.js:
        
        User Story: '{user_story}'
        
        Include specific instructions on:
        1. Necessary route endpoints and HTTP methods.
        2. Sample code snippets for Express routes and middleware.
        3. Recommended data schema using MongoDB and Mongoose.
        4. Steps for user authentication or authorization if required.
        5. Any other relevant backend logic.
        """
    else:
        raise ValueError("Mode should be either 'estimate' or 'guide'")

    #response = model.generate_content(prompt)
    #return response.text.strip()

# Clean up the extracted text
def clean_text(text):
    text = re.sub(r'\uf095', '', text)
    text = re.sub(r'-\s+', '', text)
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'\s([?.!,"])', r'\1', text)
    return text.strip()

# Further cleaning for extraneous sections
def further_clean_text(text):
    text = re.sub(r'\b(\w+)\s*-\s*(\w+)\b', r'\1\2', text)
    text = re.sub(r'3\.\d+.*Requirements.*', '', text)
    text = re.sub(r'4\.\d+.*Requirements.*', '', text)
    text = re.sub(r'5\.\d+.*Requirements.*', '', text)
    return text.strip()

# Route to handle PDF file upload and analysis
@app.route('/analyze', methods=['POST'])
def analyze_srs():
    if 'file' not in request.files or 'mode' not in request.form:
        return jsonify({"error": "Please provide a PDF file and mode (estimate/guide)"}), 400

    pdf_file = request.files['file']
    mode = request.form['mode']

    # Save the uploaded PDF to process
    pdf_path = os.path.join(app.config['UPLOAD_FOLDER'], pdf_file.filename)
    pdf_file.save(pdf_path)

    # Extract text from PDF
    text = extract_text_from_pdf(pdf_path)

    # Parse and clean user stories
    user_stories = parse_user_stories(text)
    cleaned_user_stories = {story_id: clean_text(story_text) for story_id, story_text in user_stories.items()}
    final_user_stories = {story_id: further_clean_text(story_text) for story_id, story_text in cleaned_user_stories.items()}

    # Generate responses for each user story
    results = {}
    for story_id, story_text in final_user_stories.items():
        response = generate_response(story_text, mode=mode)
        results[story_id] = response

    # Return results as JSON
    return jsonify(results)

# Run the Flask app
if __name__ == "__main__":
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with watchdog (inotify)
Traceback (most recent call last):
  File "/opt/app-root/lib64/python3.9/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/opt/app-root/lib/python3.9/site-packages/traitlets/config/application.py", line 1074, in launch_instance
    app.initialize(argv)
  File "/opt/app-root/lib/python3.9/site-packages/traitlets/config/application.py", line 118, in inner
    return method(app, *args, **kwargs)
  File "/opt/app-root/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 692, in initialize
    self.init_sockets()
  File "/opt/app-root/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 331, in init_sockets
    self.shell_port = self._bind_socket(self.shell_socket, self.shell_port)
  File "/opt/app-root/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 253, in _bind_socket
    return self._try_bind_socket(s, port)
  File "/opt/app-r

SystemExit: 1

/opt/app-root/lib64/python3.9/site-packages/IPython/core/interactiveshell.py:3558: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
